In [2]:
import os 
import tensorflow as tf

In [3]:
fnames_dataset = [
    "/Users/jonathanj/workspace/rainy-image-dataset/rain.tfrecord"
]

In [11]:
def _parse(example_proto):
    keys_to_features = {
        'image/in/filename': tf.FixedLenFeature((), tf.string, default_value=''),
        'image/out/filename': tf.FixedLenFeature((), tf.string, default_value=''),
        'image/height': tf.FixedLenFeature((), tf.int64, default_value=0),
        'image/width': tf.FixedLenFeature((), tf.int64, default_value=0),
        'image/in/contents': tf.FixedLenFeature((), tf.string, default_value=''),
        'image/out/contents': tf.FixedLenFeature((), tf.string, default_value=''),
    }
    parsed = tf.parse_single_example(example_proto, keys_to_features)
    
    def _decode(img):
        decoded = tf.image.decode_image(img)
        return decoded
        reshaped = tf.reshape(decoded, [384, 384, 1])
        return reshaped
    
    return {'image_data': _decode(parsed['image/in/contents'])}, _decode(parsed['image/out/contents'])


In [12]:
def input_fn():
    dataset = tf.data.TFRecordDataset(fnames_dataset)
    dataset = dataset.map(_parse)
    dataset = dataset.shuffle(buffer_size=1000)
    
    return dataset

In [28]:
def model_fn(features, labels, mode, params):
    inputs = features
    expected_outputs = labels
    tf.summary.image("inputs", inputs)
    tf.summary.image("expected_outputs", expected_outputs)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        pass
    else:
        raise tf.errors.UnimplementedError

# Playground

In [16]:
dataset = input_fn()
iterator = dataset.make_one_shot_iterator()
next_element = iterator.get_next()

In [18]:
elem = sess.run(next_element)

In [22]:
elem[0]['image_data'].shape

(512, 384, 3)

In [24]:
elem[1].shape

(512, 384, 3)

In [14]:
sess = tf.Session()
while True:
    try:
        sess.run(next_element)
    except tf.errors.OutOfRangeError:
        break